In [1]:
from __future__ import print_function
import sys, os
import numpy as np
from array import array
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import ROOT
from ROOT import TCanvas, TH1F, TH2F, TF1, TMath, TGraph, TFile, TSpectrum, TPaveText, TMultiGraph, TGraphErrors, TLine
from ROOT import gStyle, gROOT, gDirectory, gPad

import constants
import crv_event
import crv_spill
import utils
import root_utils
import geometry
import geometry_constants
import filepath

gROOT.Reset()
gROOT.SetBatch(1)
gROOT.ProcessLine( "gErrorIgnoreLevel = 1001;")
gStyle.SetOptStat(111111110)
gStyle.SetOptFit(0)
gStyle.SetLineScalePS(0.3)

analysis_dir = "~/CRVteststand/analysis/testbench/"
analysisFileName = analysis_dir + "temperatureScanAnalysis.root"

analysisFile = TFile(analysisFileName, "READ")
spilltree = analysisFile.Get("spill_summary")
subruntree = analysisFile.Get('subrun_summary')
nSpill = spilltree.GetEntries()
nSubrun = subruntree.GetEntries()
nCh = geometry_constants.nChannelPerFEB

PEsRaw = {"cosmics":[[[] for j in range(nCh)] for i in range(2)],
          "LED":[[[] for j in range(nCh)] for i in range(2)]}
PEsCorrected = {"cosmics":[[[] for j in range(nCh)] for i in range(2)],
                "LED":[[[] for j in range(nCh)] for i in range(2)]}
CMBtemp = {"cosmics":[[[] for j in range(nCh)] for i in range(2)], 
           "LED":[[[] for j in range(nCh)] for i in range(2)]}

for iSubrun in range(nSubrun):
    subruntree.GetEntry(iSubrun)
    tType = subruntree.runType[0]
    
    if tType == "cosmics":
        continue
    
    for iFEB in range(2):
        for iCh in range(nCh):
            CMBtemp[tType][iFEB][iCh].append(subruntree.temperatureCMB[iFEB*nCh+iCh])
            
            tspec = analysisFile.Get(subruntree.spectrum[iFEB*nCh+iCh]).Clone()
            funcspec = TF1("funcSpec_run%i_%i_FEB%i_Ch%i"%(subruntree.runNum, subruntree.subrunNum, iFEB, iCh),"[0]*TMath::Gaus(x,[1],[2],1)",40.,100.)
            funcspec.SetParameter(0,tspec.GetEntries())
            funcspec.SetParameter(1,tspec.GetMean())
            funcspec.SetParameter(2,tspec.GetRMS()/2.)
            funcspec.SetParName(0, "A")
            funcspec.SetParName(1, "#mu")
            funcspec.SetParName(2, "#sigma")
            funcspec.SetParLimits(0, tspec.GetEntries()/20., 10.*tspec.GetEntries())
            funcspec.SetParLimits(1, tspec.GetMean()-2.*tspec.GetRMS(), tspec.GetMean()+2.*tspec.GetRMS())
            funcspec.SetParLimits(2, .005, 30)
            fitoption = "QRS"
            if tspec.GetEntries() < 2000:
                fitoption+="L"
            fitmin = tspec.GetMean()-2.*tspec.GetRMS()
            fitmax = tspec.GetMean()+2.*tspec.GetRMS()
            frp1 = tspec.Fit(funcspec,fitoption,"",fitmin,fitmax)
            PEsRaw[tType][iFEB][iCh].append(funcspec.GetParameter(1))
                       
            tspecCorrected = analysisFile.Get(subruntree.spectrumCorrected[iFEB*nCh+iCh]).Clone()
            funcspecCorrected = TF1("funcSpecCorrected_run%i_%i_FEB%i_Ch%i"%(subruntree.runNum, subruntree.subrunNum, iFEB, iCh),"[0]*TMath::Gaus(x,[1],[2],1)",40.,100.)
            funcspecCorrected.SetParameter(0,tspecCorrected.GetEntries())
            funcspecCorrected.SetParameter(1,tspecCorrected.GetMean())
            funcspecCorrected.SetParameter(2,tspecCorrected.GetRMS()/2.)
            funcspecCorrected.SetParName(0, "A")
            funcspecCorrected.SetParName(1, "#mu")
            funcspecCorrected.SetParName(2, "#sigma")
            funcspecCorrected.SetParLimits(0, tspecCorrected.GetEntries()/20., 10.*tspecCorrected.GetEntries())
            funcspecCorrected.SetParLimits(1, tspecCorrected.GetMean()-2.*tspecCorrected.GetRMS(), tspecCorrected.GetMean()+2.*tspecCorrected.GetRMS())
            funcspecCorrected.SetParLimits(2, .005, 30)
            fitoption = "QRS"
            if tspecCorrected.GetEntries() < 2000:
                fitoption+="L"
            fitmin = tspecCorrected.GetMean()-2.*tspecCorrected.GetRMS()
            fitmax = tspecCorrected.GetMean()+2.*tspecCorrected.GetRMS()
            frp2 = tspecCorrected.Fit(funcspecCorrected,fitoption,"",fitmin,fitmax)
            PEsCorrected[tType][iFEB][iCh].append(funcspecCorrected.GetParameter(1))
            
            del funcspec
            del tspec
            del funcspecCorrected
            del tspecCorrected
            
    if iSubrun%20 == 0:
        print("Processed %i/%i"%(iSubrun, nSubrun))
            
# put things into scatter plots, then fit LED graphs
xdiv = 2
ydiv = 1

pdfname = analysis_dir+"lightYield.pdf"
fC00 = TCanvas("c00", "c00", 1200*xdiv, 900*ydiv)
fC00.Divide(xdiv,ydiv,0.0001,0.001)
fC00.Print(pdfname+"[", "pdf")

PEs_CMBtemp = [[None]*nCh for i in range(2)]
PEsCorrected_CMBtemp = [[None]*nCh for i in range(2)]

for iFEB in range(2):
    for iCh in range(nCh):
        fCanvas= TCanvas("c_%i_%i"%(iFEB, iCh), "c_%i_%i"%(iFEB, iCh), 1200*xdiv, 900*ydiv)
        fCanvas.Divide(xdiv,ydiv,0.0001,0.001)
        gStyle.SetOptStat(0)
        gStyle.SetOptFit(111)
        
        fCanvas.cd(1)
        PEs_CMBtemp[iFEB][iCh] = TGraph(len(CMBtemp['LED'][iFEB][iCh]), array('d', CMBtemp['LED'][iFEB][iCh]), array('d', PEsRaw['LED'][iFEB][iCh]))
        PEs_CMBtemp[iFEB][iCh].SetName("scat_PEs_CMBtemp_%i_%i_LED"%(iFEB, iCh))
        PEs_CMBtemp[iFEB][iCh].SetTitle("Raw PEs w.r.t. CMB Temperature; CMB Temperature [#circC]; Light Yield (Raw) [PE]")
        PEs_CMBtemp[iFEB][iCh].SetMarkerStyle(21)
        PEs_CMBtemp[iFEB][iCh].SetMarkerColor(2)
        PEs_CMBtemp[iFEB][iCh].SetLineColor(2)
        PEs_CMBtemp[iFEB][iCh].SetLineWidth(3)
        PEs_CMBtemp[iFEB][iCh].SetMarkerSize(1)
        PEs_CMBtemp[iFEB][iCh].Draw("AP")
        
        fLegend1 = ROOT.TLegend(0.75, 0.6, 0.98, 0.72)
        fLegend1.SetHeader("FEB%i_Ch%02i"%(iFEB, iCh),"C")
        fLegend1.AddEntry(PEs_CMBtemp[iFEB][iCh], "LED runs")
        fLegend1.Draw("SAME")
        
        fCanvas.cd(2)
        PEsCorrected_CMBtemp[iFEB][iCh] = TGraph(len(CMBtemp['LED'][iFEB][iCh]), array('d', CMBtemp['LED'][iFEB][iCh]), array('d', PEsCorrected['LED'][iFEB][iCh]))
        PEsCorrected_CMBtemp[iFEB][iCh].SetName("scat_PEsCorrected_CMBtemp_%i_%i_LED"%(iFEB, iCh))
        PEsCorrected_CMBtemp[iFEB][iCh].SetTitle("Corrected PEs w.r.t. CMB Temperature; CMB Temperature [#circC]; Light Yield (Temp. Corrected) [PE]")
        PEsCorrected_CMBtemp[iFEB][iCh].SetMarkerStyle(21)
        PEsCorrected_CMBtemp[iFEB][iCh].SetMarkerColor(2)
        PEsCorrected_CMBtemp[iFEB][iCh].SetLineColor(2)
        PEsCorrected_CMBtemp[iFEB][iCh].SetLineWidth(3)
        PEsCorrected_CMBtemp[iFEB][iCh].SetMarkerSize(1)
        PEsCorrected_CMBtemp[iFEB][iCh].Draw("AP")
        
        fCanvas.Print(pdfname, "Title: FEB%i_Ch%02i"%(iFEB, iCh))
        
        
analysisFile.Close()

fC00.Print(pdfname+"]", "pdf")

from IPython.display import IFrame
IFrame("lightYield.pdf", width=1200, height=800)
        

Welcome to JupyROOT 6.28/00
Processed 20/458
Processed 40/458
Processed 60/458
Processed 80/458
Processed 100/458
Processed 120/458
Processed 140/458
Processed 160/458
Processed 180/458
Processed 200/458
Processed 220/458
Processed 240/458
Processed 260/458
Processed 280/458
Processed 300/458
Processed 320/458
Processed 340/458
Processed 360/458
Processed 380/458
Processed 400/458
Processed 420/458
Processed 440/458
